In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from datetime import datetime

In [2]:
df = pd.read_csv('data.csv')

In [3]:
df.head()

,CustomerID,Name,Age,MonthlyBill,DataUsageGB,ContractMonths,DevicesOwned,Exited,PhoneNumber,EmailAddress,City,State,ZipCode,PlanType,TotalSpent,CustomerSince,LastInteraction
0,1616ab20-0dd5-43bb-a57c-62b57a4d3886,Karen Wilson,29,195.70,4.53,12,2,1,635-202-1002x5152,samanthasparks@example.org,North Donna,Kentucky,16336,Standard,612.42,2021-11-02,2023-01-30
1,0d46fa0e-7bb9-4338-9633-90e2731c69f6,Haley Benson,25,100.71,6.32,12,5,0,001-990-507-9036x985,justintucker@example.com,Lake Ashley,South Dakota,1958,Premium,528.82,2020-04-05,2023-01-04
2,15849c43-75e3-46fa-b1ec-e589758552c3,Troy Friedman,63,179.56,8.44,24,2,0,(755)329-2708x74703,alyssa08@example.net,Johnsonmouth,Delaware,39715,Premium,710.03,2019-05-22,2022-11-16
3,febd1d22-6358-4b2d-9a4e-f33e5833391d,Lisa Moore,62,121.10,3.81,24,1,0,+1-900-342-7352x40670,amandawhitehead@example.com,East Anthonybury,Alabama,87890,Basic,119.19,2020-10-24,2023-02-21
4,58a0a76b-2a68-45ed-b512-b64a31e5094a,Ryan Garrison,27,108.99,6.59,36,3,1,(351)367-6820x10664,wcollins@example.com,Williamston,South Carolina,79380,Basic,642.76,2021-06-17,2023-04-15


In [4]:
# Preprocessing
df = df.drop(columns=['CustomerID', 'Name', 'PhoneNumber', 'EmailAddress', 'City'])

In [5]:
df.head()

,Age,MonthlyBill,DataUsageGB,ContractMonths,DevicesOwned,Exited,State,ZipCode,PlanType,TotalSpent,CustomerSince,LastInteraction
0,29,195.70,4.53,12,2,1,Kentucky,16336,Standard,612.42,2021-11-02,2023-01-30
1,25,100.71,6.32,12,5,0,South Dakota,1958,Premium,528.82,2020-04-05,2023-01-04
2,63,179.56,8.44,24,2,0,Delaware,39715,Premium,710.03,2019-05-22,2022-11-16
3,62,121.10,3.81,24,1,0,Alabama,87890,Basic,119.19,2020-10-24,2023-02-21
4,27,108.99,6.59,36,3,1,South Carolina,79380,Basic,642.76,2021-06-17,2023-04-15


In [6]:
## Converting PlanType, State, and ZipCode to numerical
le = LabelEncoder()
df['PlanType'] = le.fit_transform(df['PlanType'])
df['State'] = le.fit_transform(df['State'])
df['ZipCode'] = le.fit_transform(df['ZipCode'])

In [7]:
## Converting date columns to datetime
df['CustomerSince'] = pd.to_datetime(df['CustomerSince'])
df['LastInteraction'] = pd.to_datetime(df['LastInteraction'])

In [8]:
## Calculating total months since customers joined and last interacted with the company
df['MonthsCustomerSince'] = (datetime.now() - df['CustomerSince']).dt.days // 30
df['MonthsSinceLastInteraction'] = (datetime.now() - df['LastInteraction']).dt.days // 30

In [9]:
## Features vs target variable
x = df[['Age', 'MonthlyBill', 'DataUsageGB', 'ContractMonths', 'DevicesOwned', 'State', 'PlanType', 'TotalSpent', 'MonthsCustomerSince', 'MonthsSinceLastInteraction']]
y = df['Exited']

In [10]:
# Split to train & test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=30)

In [11]:
model = LogisticRegression()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.49      0.60      0.54       292
           1       0.52      0.42      0.46       306

    accuracy                           0.50       598
   macro avg       0.51      0.51      0.50       598
weighted avg       0.51      0.50      0.50       598



/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [12]:
model2 = XGBClassifier()
model2.fit(x_train, y_train)
y2_pred = model2.predict(x_test)
print(classification_report(y_test, y2_pred))

              precision    recall  f1-score   support

           0       0.48      0.51      0.49       292
           1       0.50      0.47      0.48       306

    accuracy                           0.49       598
   macro avg       0.49      0.49      0.49       598
weighted avg       0.49      0.49      0.49       598

